1. Manually take confocal image at specified coordinates.
2. Use diff_register code to extract same image from tilescan image.
3. Convert both images to same coordinates.
4. Compare the images using some algorithm (here: https://www.pyimagesearch.com/2014/09/15/python-compare-two-images/)